In [1]:
import os

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']="1" # -1 == cpu, 0,1 == gpus 0,1
import math
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import skimage.io
import skimage.transform
import skimage.filters
import keras
from keras.applications import vgg16
from keras import backend as K
from keras.models import load_model
from keras.models import model_from_json
import cv2
# Allow graph embeding in notebook
%matplotlib inline

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
def tensor_summary(tensor):
    """Display shape, min, and max values of a tensor."""
    print("shape: {}  min: {}  max: {}".format(tensor.shape, tensor.min(), tensor.max()))

    
def normalize(image):
    """Takes a tensor of 3 dimensions (height, width, colors) and normalizes it's values
    to be between 0 and 1 so it's suitable for displaying as an image."""
    image = image.astype(np.float32)
    return (image - image.min()) / (image.max() - image.min() + 1e-5)


def display_images(images, titles=None, cols=5, interpolation=None, cmap="Greys_r"):
    """
    images: A list of images. I can be either:
        - A list of Numpy arrays. Each array represents an image.
        - A list of lists of Numpy arrays. In this case, the images in
          the inner lists are concatentated to make one image.
    """
    titles = titles or [""] * len(images)
    rows = math.ceil(len(images) / cols)
    height_ratio = 1.2 * (rows/cols) * (0.5 if type(images[0]) is not np.ndarray else 1)
    plt.figure(figsize=(11, 11 * height_ratio))
    i = 1
    for image, title in zip(images, titles):
        plt.subplot(rows, cols, i)
        plt.axis("off")
        # Is image a list? If so, merge them into one image.
        if type(image) is not np.ndarray:
            image = [normalize(g) for g in image]
            image = np.concatenate(image, axis=1)
        else:
            image = normalize(image)
        plt.title(title, fontsize=9)
        plt.imshow(image, cmap=cmap, interpolation=interpolation)
        i += 1

In [3]:
def build_backprop(model, loss):
    # Gradient of the input image with respect to the loss function
    gradients = K.gradients(loss, model.input)[0]
    # Normalize the gradients
    gradients /= (K.sqrt(K.mean(K.square(gradients))) + 1e-5)
    # Keras function to calculate the gradients and loss
    return K.function([model.input], [loss, gradients])

In [4]:
#model  = load_model('/home/user/keras/examples/mnist.h5')
with open('modelcfg.json') as fi:
    json_str = json.load(fi)
model = model_from_json(json.dumps(json_str))
model.load_weights('weights.hdf5')

In [5]:
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 22, 22, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 22, 22, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
activation_3 (Activation)    (None, 11, 11, 64)        0         
__________

In [22]:
label_map = {1: 'a', 2: 'h', 3: 'm', 4: 'o', 5: 's', 0: 'T'}
name = 'Thomas'
reverse_map = {}

In [7]:
# Iteratively apply gradient ascent
def get_image(random_image):
    for i in range(500):
        loss, grads = backprop([random_image])

        # Multiply gradients by the learning rate and add to the image
        # Optionally, apply a gaussian filter to the gradients to smooth
        # out the generated image. This gives better results.
        # The first line, which is commented out, is the native method
        # and the following line uses the filter. Try with both to
        # see the difference.
        #
        random_image += grads * .1
        #random_image += skimage.filters.gaussian(np.clip(grads, -1, 1), 2) 

        # Print loss value
        if i % 100 == 0:
            print('Loss:', loss)
    predictions = model.predict(random_image)
    print predictions
    return random_image

In [27]:
def save_image(im_name,image):
    factor = 16
    scaled = np.repeat(image,factor,axis=1)
    scaled = np.repeat(scaled,factor,axis=2)
    im = np.squeeze(scaled)*255
    im = im.astype(np.uint8)
    cv2.imwrite(im_name+'.tif',im)
    bw = cv2.cvtColor(im,cv2.COLOR_RGB2GRAY)
    cv2.imwrite(im_name+'_bw.tif',bw)
    return im,bw

In [28]:
for i in range(6):
    random_image = np.random.random((1, 24, 24, 3))
    loss_function = K.mean(model.get_layer("predictions").output[:,i])
    backprop = build_backprop(model, loss_function)
    image = get_image(random_image)
    reverse_map[label_map[i]] = save_image(label_map[i],image)

('Loss:', 0.28237203)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
[[  1.00000000e+00   3.89690891e-15   2.37479790e-18   4.79984275e-10
    3.84357232e-13   4.08063967e-15]]
('Loss:', 0.0040964135)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
[[  2.49515544e-14   1.00000000e+00   1.01154453e-15   2.69899761e-25
    8.02959519e-14   8.96402830e-12]]
('Loss:', 0.027503045)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
[[  6.39649127e-14   1.86542351e-10   1.00000000e+00   1.10252377e-13
    7.61091259e-11   1.51009695e-12]]
('Loss:', 0.0066232225)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
[[  2.98545211e-09   1.36589268e-10   2.28166730e-09   1.00000000e+00
    3.09247489e-10   3.70609576e-14]]
('Loss:', 0.56726795)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
('Loss:', 1.0)
[[  2.46139636e-10   6.50712018e-17   1.07753673e-09   8.62956733e-13
    1.00000000e+00   2.34437969e-10]]
('Loss:', 0.0084030842)
('Loss:', 1.0)
('Loss

In [38]:
stacked = np.hstack([reverse_map[letter][0] for letter in name])

In [39]:
cv2.imwrite('stacked.tif',stacked)

True